In [1]:
import sddk
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
import getpass
from github import Github
import requests

In [2]:
g = Github("kasev", getpass.getpass("github password: "))

In [3]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


In [4]:
LAGT = s.read_file("SDAM_data/AGT/AGT_raw_20240107.json")

In [5]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr


# PROIEL - the GNT reveals uncomplete...

In [6]:
resp = requests.get("https://raw.githubusercontent.com/proiel/proiel-treebank/master/greek-nt.xml")
soup = BeautifulSoup(resp.text, "html.parser")

In [7]:
divs = soup.find_all("div")

In [25]:
len(soup.find_all("sentence"))

11505

In [39]:
failed = []
div_sents = []
for div in divs: 
    div_title = div.title.get_text()
    for sent in div.find_all("sentence"):
        tokens = [t for t in sent.find_all("token")]
        try:
            sent_lemmata = [t["lemma"].partition("#")[0] for t in tokens if t["part-of-speech"][0] in ["N", "V", "A"]]
            div_sents.append((div_title, sent_lemmata))
        except:
            sent_lemmata = []
            for t in tokens:
                try:
                    if t["part-of-speech"][0] in ["N", "V", "A"]:
                        sent_lemmata.append(t["lemma"].partition("#")[0])
                except:
                    pass
            div_sents.append((div_title, sent_lemmata))

In [42]:
proiel_sents = pd.DataFrame(div_sents)
proiel_sents.columns = ["chapter", "sentence"]
proiel_sents.sample(10)

,chapter,sentence
3451,Luke 7,"[ὅμοιος, εἰμί, παιδίον, ἀγορά, κάθημαι, προσφω..."
7670,Acts 25,"[Φῆστος, ἐπιβαίνω, ἐπαρχεῖος, ἡμέρα, ἀναβαίνω,..."
8447,Romans 15,"[εὐδοκέω, ὀφειλέτης, εἰμί]"
1886,Mark 1,"[εἰσπορεύομαι, Καφαρναούμ]"
6870,Acts 8,"[ἀνοίγνυμι, Φίλιππος, στόμα, ἄρχω, γραφή, εὐαγ..."
2922,Mark 15,[λαμβάνω]
11216,Revelation 14,"[ὁράω, ἄλλος, ἄγγελος, πέτομαι, μεσουράνημα, ἔ..."
10579,Hebrews 10,[ἀνταποδίδωμι]
2935,Mark 15,"[γίγνομαι, ὥρα, σκότος, γίγνομαι, ὅλος, γῆ, ὥρα]"
389,Matthew 8,"[δειλός, εἰμί, ὀλιγόπιστος]"


In [63]:
proiel_sents.head(20)

,chapter,sentence,book
0,Matthew 1,"[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυίδ...",Matthew
1,Matthew 1,"[Ἀβραάμ, γεννάω, Ἰσαάκ]",Matthew
2,Matthew 1,"[Ἰσαάκ, γεννάω, Ἰακώβ]",Matthew
3,Matthew 1,"[Ἰακώβ, γεννάω, Ἰούδας, ἀδελφός]",Matthew
4,Matthew 1,"[Ἰούδας, γεννάω, Φάρες, Ζάρα, Θαμάρ]",Matthew
5,Matthew 1,"[Φάρες, γεννάω, Ἑσρώμ]",Matthew
6,Matthew 1,"[Ἑσρώμ, γεννάω, Ἀράμ]",Matthew
7,Matthew 1,"[Ἀράμ, γεννάω, Ἀμιναδάβ]",Matthew
8,Matthew 1,"[Ἀμιναδάβ, γεννάω, Ναασσών]",Matthew
9,Matthew 1,"[Ναασσών, γεννάω, Σαλμών]",Matthew


In [43]:
proiel_sents["book"] = proiel_sents["chapter"].apply(lambda x: x.rpartition(" ")[0])
set(proiel_sents["book"])

{'1 Corinthians',
 '1 Peter',
 '1 Thessalonians',
 '1 Timothy',
 '2 Corinthians',
 '2 Thessalonians',
 '2 Timothy',
 '3 John',
 'Acts',
 'Colossians',
 'Ephesians',
 'Galatians',
 'Hebrews',
 'James',
 'John',
 'Jude',
 'Luke',
 'Mark',
 'Matthew',
 'Philemon',
 'Philippians',
 'Revelation',
 'Romans',
 'Titus'}

Ok, some books are missing here! Such as 1 John and 2 John 

# MorphGNT

In [45]:
resp = requests.get("https://github.com/morphgnt/sblgnt")

In [47]:
resp.content

b'\n\n\n\n\n\n\n<!DOCTYPE html>\n<html\n  lang="en"\n  \n  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"\n  data-a11y-animated-images="system" data-a11y-link-underlines="true"\n  >\n\n\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-38f1bf52eeeb.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-56010aa53a8f.css" /><link data-color-theme="dark_dimm

In [89]:
repo = g.get_user("morphgnt").get_repo("sblgnt" )
contents = repo.get_contents("")
filenames = [content.path for content in contents if ".txt" in content.path]

In [91]:
len(filenames)

27

In [92]:
base_url = "https://raw.githubusercontent.com/morphgnt/sblgnt/master/"
# test with individual book
book = pd.read_csv(base_url + filenames[0], delimiter=" ", header=None)

In [94]:
book.head(10)

,0,1,2,3,4,5,6
0,10101,N-,----NSF-,Βίβλος,Βίβλος,βίβλος,βίβλος
1,10101,N-,----GSF-,γενέσεως,γενέσεως,γενέσεως,γένεσις
2,10101,N-,----GSM-,Ἰησοῦ,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς
3,10101,N-,----GSM-,χριστοῦ,χριστοῦ,Χριστοῦ,Χριστός
4,10101,N-,----GSM-,υἱοῦ,υἱοῦ,υἱοῦ,υἱός
5,10101,N-,----GSM-,Δαυὶδ,Δαυὶδ,Δαυίδ,Δαυίδ
6,10101,N-,----GSM-,υἱοῦ,υἱοῦ,υἱοῦ,υἱός
7,10101,N-,----GSM-,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,Ἀβραάμ
8,10102,N-,----NSM-,Ἀβραὰμ,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ
9,10102,V-,3AAI-S--,ἐγέννησεν,ἐγέννησεν,ἐγέννησε(ν),γεννάω


In [115]:
word = "fdfdfd;"
if re.search(word[-1], "\·|\."):
    print(word)

In [127]:
book = pd.read_csv(base_url + filenames[0], delimiter=" ", header=None)
book.columns = ["verse", "pos", "parsing", "word_punct", "word", "norm", "lemma"]
book_sents = []
sent_lemmata = []
raw_sent = ""
for word, pos, lemma in zip(book["word_punct"], book["pos"], book["lemma"]):
    raw_sent += word + " "
    if pos[0] in ["N", "A", "V"]:
        sent_lemmata.append(lemma)
    if re.search("[\·\.\;\!]", word[-1]):
        book_sents.append(sent_lemmata) # (raw_sent, sent_lemmata))
        sent_lemmata = []
        raw_sent = ""

In [128]:
len(book_sents)

1679

In [138]:
books_sents = []
for filename in filenames:
    book = pd.read_csv(base_url + filename, delimiter=" ", header=None)
    book.columns = ["verse", "pos", "parsing", "word_punct", "word", "norm", "lemma"]
    book_sents = [] 
    sent_lemmata = []
    raw_sent = ""
    for word, pos, lemma in zip(book["word_punct"], book["pos"], book["lemma"]):
        raw_sent += word + " "
        if pos[0] in ["N", "A", "V"]:
            sent_lemmata.append(lemma)
        if re.search("[\·\.\;\!]", word[-1]):
            book_sents.append(sent_lemmata) # (raw_sent, sent_lemmata))
            sent_lemmata = []
            raw_sent = ""
    books_sents.append({"filename": filename, "lemmatized_sentences" : book_sents})

In [139]:
len(books_sents)

27

In [140]:
morphgnt_dict = dict(zip(LAGT[LAGT["doc_id"].str.startswith("tlg0031")]["doc_id"].tolist(), books_sents))

# LXX

In [150]:
LAGT[LAGT["author"].str.contains("Septuagint", na=False)]

,author_id,doc_id,filename,author,title,string,wordcount,source
347,tlg0527,tlg0527.tlg001,tlg0527.tlg001.opp-grc2.xml,Septuaginta,Genesis,ΕΝ APXH ἐποίησεν ὁ θεὸς τὸν οὐρανὸν καὶ τὴν γῆ...,46422,1Kgr
348,tlg0527,tlg0527.tlg002,tlg0527.tlg002.opp-grc2.xml,Septuaginta,Exodus,ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ τῶν εἰσπεπορε...,36962,1Kgr
349,tlg0527,tlg0527.tlg003,tlg0527.tlg003.opp-grc2.xml,Septuaginta,Leviticus,KAI ἀνεκάλεσεν Μωυσῆν καὶ ἐλάλησεν Κύριος αὐτῷ...,27730,1Kgr
350,tlg0527,tlg0527.tlg004,tlg0527.tlg004.opp-grc2.xml,Septuaginta,Numeri,KAI ἐλάλησεν κύριος πρὸς Μωυσῆν ἐν τῇ ἐρήμῳ τῇ...,36189,1Kgr
351,tlg0527,tlg0527.tlg005,tlg0527.tlg005.opp-grc2.xml,Septuaginta,Deuteronomium,OYTOI οἱ λόγοι οὓς ἐλάλησεν Μωυσῆς παντὶ Ἰσραὴ...,33651,1Kgr
352,tlg0527,tlg0527.tlg006,tlg0527.tlg006.opp-grc2.xml,Septuaginta,Josue (Cod. Vaticanus + Cod. Alexandrinus),ΚΑΙ ἐγένετο μετὰ τὴν τελευτὴν Μωυσῆ εἶπεν κύρι...,24806,1Kgr
353,tlg0527,tlg0527.tlg008,tlg0527.tlg008.opp-grc2.xml,Septuaginta,Judices (Cod. Alexandrinus),ΚΑΙ ἐγένετο μετὰ τὴν τελευτὴν Ἰησοῦ καὶ ἐπηρώτ...,28640,1Kgr
354,tlg0527,tlg0527.tlg010,tlg0527.tlg010.opp-grc2.xml,Septuaginta,Ruth,ΚΑΙ ἐγένετο ἐν τῷ κρίνειν τοὺς κριτὰς καὶ ἐγέν...,2681,1Kgr
355,tlg0527,tlg0527.tlg011,tlg0527.tlg011.opp-grc2.xml,Septuaginta,Regnorum I (Samuelis I in textu Masoretico),ἍΝΘΡΩΠΟΣ ἦν ἐξ ῾Αρμαθάιμ Σειφὰ ἐξ ὄρους ᾿Εφράι...,28201,1Kgr
356,tlg0527,tlg0527.tlg012,tlg0527.tlg012.opp-grc2.xml,Septuaginta,Regnorum II (Samuelis II In Textu Masoretico),"ΚΑΙ ἐγένετο μετὰ τὸ ἀποθανεῖν Σαοὐλ, καὶ Δαυεὶ...",23848,1Kgr


In [153]:
repo = g.get_user("nathans").get_repo("lxxmorph-unicode")
contents = repo.get_contents("")
filenames = [content.path for content in contents if re.search("^\d\d\.", content.path)]
filenames[:10]

['01.Gen.txt',
 '02.Exod.txt',
 '03.Lev.txt',
 '04.Num.txt',
 '05.Deut.txt',
 '06.JoshB.txt',
 '07.JoshA.txt',
 '08.JudgesB.txt',
 '09.JudgesA.txt',
 '10.Ruth.txt']

In [156]:
base_url = "https://raw.githubusercontent.com/nathans/lxxmorph-unicode/master/"
lines = requests.get(base_url + filenames[0]).text.split("\n")

In [158]:
lines[:15]

['Gen 1:1',
 'ἐν P--------- ἐν',
 'ἀρχῇ N1--DSF--- ἀρχή',
 'ἐποίησεν VAI-AAI3S- ποιέω',
 'ὁ RA--NSM--- ὁ',
 'θεὸς N2--NSM--- θεός',
 'τὸν RA--ASM--- ὁ',
 'οὐρανὸν N2--ASM--- οὐρανός',
 'καὶ C--------- καί',
 'τὴν RA--ASF--- ὁ',
 'γῆν N1--ASF--- γῆ',
 '',
 'Gen 1:2',
 'ἡ RA--NSF--- ὁ',
 'δὲ X--------- δέ']

In [ ]:
books_sents = []
for filename in filenames:
    book = pd.read_csv(base_url + filename, delimiter=" ", header=None)
    book.columns = ["verse", "pos", "parsing", "word_punct", "word", "norm", "lemma"]
    book_sents = []
    sent_lemmata = []
    raw_sent = ""
    for word, pos, lemma in zip(book["word_punct"], book["pos"], book["lemma"]):
        raw_sent += word + " "
        if pos[0] in ["N", "A", "V"]:
            sent_lemmata.append(lemma)
        if re.search("[\·\.\;\!]", word[-1]):
            book_sents.append(sent_lemmata) # (raw_sent, sent_lemmata))
            sent_lemmata = []
            raw_sent = ""
    books_sents.append({"filename": filename, "lemmatized_sentences" : book_sents})

# GLAUx

In [ ]:
div_sents

In [6]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("perseids-publications").get_repo("glaux-trees" )
paths = []
contents = repo.get_contents("public/xml")
while contents:
    file_content = contents.pop(0)
    path_and_name = file_content.path
    if ".xml" in str(path_and_name):
        paths.append(path_and_name)

CPU times: user 35.1 ms, sys: 6.5 ms, total: 41.6 ms
Wall time: 1.84 s


In [7]:
len(paths)

936

In [8]:
paths[0]

'public/xml/0001-001.xml'

In [9]:
#test
response = requests.get('https://raw.githubusercontent.com/perseids-publications/glaux-trees/master/' + paths[0])
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()[:1000])

<treebank direction="ltr" format="pedalion" version="2" xml:lang="grc" xmlns:saxon="http://saxon.sf.net/">
 <sentence document_id="0001-001" id="1" subdoc="1">
  <word form="ἀρχόμενος" head="640963" id="640955" lemma="ἄρχω" line="1" postag="v-sppemn-" relation="ADV">
  </word>
  <word form="σέο" head="640955" id="640956" lemma="σύ" line="1" postag="p-s---cg-" relation="OBJ">
  </word>
  <word form="," head="640955" id="640957" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="Φοῖβε" head="640963" id="640958" lemma="Φοῖβος" line="1" postag="n-s---mv-" relation="ExD">
  </word>
  <word form="," head="640955" id="640959" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="παλαιγενέων" head="640962" id="640960" lemma="παλαιγενής" line="1" postag="a-p---mg-" relation="ATR">
  </word>
  <word form="κλέα" head="640963" id="640961" lemma="κλέος" line="1" postag="n-p---na-" relation="SBJ">
  </word>
  <word form="φωτῶν" head="640961" id=

In [10]:
def from_filename_to_docid(filename):
    filename = filename.rpartition("/")[2]
    parts = re.split("\-|\.", filename)[:2]
    docid = "tlg"+parts[0] + ".tlg" + parts[1]
    return docid
glaux_docids = [from_filename_to_docid(filename) for filename in paths]
glaux_docids[:10]

['tlg0001.tlg001',
 'tlg0002.tlg004',
 'tlg0003.tlg001',
 'tlg0003.tlg002',
 'tlg0005.tlg001',
 'tlg0005.tlg002',
 'tlg0005.tlg005',
 'tlg0006.tlg007',
 'tlg0006.tlg008',
 'tlg0006.tlg009']

In [11]:
workid_filename_dict = dict([(from_filename_to_docid(filename), filename)  for filename in paths])

In [12]:
len(LAGT)

1892

In [13]:
len(set(LAGT["doc_id"]) & set(glaux_docids))

862

### GLAUx simple test with one work

In [14]:
base_url = "https://raw.githubusercontent.com/perseids-publications/glaux-trees/master/"

In [15]:
response = requests.get(base_url + paths[0])
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()[:5000])

<treebank direction="ltr" format="pedalion" version="2" xml:lang="grc" xmlns:saxon="http://saxon.sf.net/">
 <sentence document_id="0001-001" id="1" subdoc="1">
  <word form="ἀρχόμενος" head="640963" id="640955" lemma="ἄρχω" line="1" postag="v-sppemn-" relation="ADV">
  </word>
  <word form="σέο" head="640955" id="640956" lemma="σύ" line="1" postag="p-s---cg-" relation="OBJ">
  </word>
  <word form="," head="640955" id="640957" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="Φοῖβε" head="640963" id="640958" lemma="Φοῖβος" line="1" postag="n-s---mv-" relation="ExD">
  </word>
  <word form="," head="640955" id="640959" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="παλαιγενέων" head="640962" id="640960" lemma="παλαιγενής" line="1" postag="a-p---mg-" relation="ATR">
  </word>
  <word form="κλέα" head="640963" id="640961" lemma="κλέος" line="1" postag="n-p---na-" relation="SBJ">
  </word>
  <word form="φωτῶν" head="640961" id=

In [16]:
#to obtain ordinary sentence data... (not used here)
sentences = []
for sent in soup.find_all("sentence"):
    sentence = ""
    for w in sent.find_all("word"):
        wform = w["form"]
        if w["relation"]=="PUNCT":
            sentence += wform
        else:
            sentence += " " + wform
    sentences.append(sentence[1:])
sentences

['ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ.',
 'τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδ’ ἀνέρος, ὅντιν’ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπ’ ἐννεσίῃσι δαμῆναι.',
 'δηρὸν δ’ οὐ μετέπειτα τεὴν κατὰ βάξιν Ἰήσων χειμερίοιο ῥέεθρα κιὼν διὰ ποσσὶν Ἀναύρου ἄλλο μὲν ἐξεσάωσεν ὑπ’ ἰλύος, ἄλλο δ’ ἔνερθεν κάλλιπεν αὖθι πέδιλον ἐνισχόμενον προχοῇσιν.',
 'ἵκετο δ’ ἐς Πελίην αὐτοσχεδὸν ἀντιβολήσων εἰλαπίνης, ἣν πατρὶ Ποσειδάωνι καὶ ἄλλοις ῥέζε θεοῖς, Ἥρης δὲ Πελασγίδος οὐκ ἀλέγιζεν.',
 'αἶψα δὲ τόνγ’ ἐσιδὼν ἐφράσσατο, καί οἱ ἄεθλον ἔντυε ναυτιλίης πολυκηδέος, ὄφρ’ ἐνὶ πόντῳ ἠὲ καὶ ἀλλοδαποῖσι μετ’ ἀνδράσι νόστον ὀλέσσῃ.',
 'νῆα μὲν οὖν οἱ πρόσθεν ἐπικλείουσιν ἀοιδοὶ Ἄργον Ἀθηναίης καμέειν ὑποθημοσύνῃσιν.',
 'νῦν δ’ ἂν ἐγὼ γενεήν τε καὶ οὔνομα μυθησαίμην ἡρώων, δολιχῆς τε πόρους ἁλός, ὅσσα τ’ ἔρεξαν πλαζόμενοι·',
 'Μοῦσαι δ’ ὑποφήτορες εἶεν ἀοιδῆς.',
 'π

In [17]:
paths[0]

'public/xml/0001-001.xml'

In [18]:
LAGT["lemmatized_sentences"] = None# LAGT.apply(lambda row: [], axis=1)

In [19]:
workid_filename_dict

{'tlg0001.tlg001': 'public/xml/0001-001.xml',
 'tlg0002.tlg004': 'public/xml/0002-004.xml',
 'tlg0003.tlg001': 'public/xml/0003-001.xml',
 'tlg0003.tlg002': 'public/xml/0003-002.xml',
 'tlg0005.tlg001': 'public/xml/0005-001.xml',
 'tlg0005.tlg002': 'public/xml/0005-002.xml',
 'tlg0005.tlg005': 'public/xml/0005-005.xml',
 'tlg0006.tlg007': 'public/xml/0006-007.xml',
 'tlg0006.tlg008': 'public/xml/0006-008.xml',
 'tlg0006.tlg009': 'public/xml/0006-009.xml',
 'tlg0006.tlg010': 'public/xml/0006-010.xml',
 'tlg0006.tlg011': 'public/xml/0006-011.xml',
 'tlg0006.tlg012': 'public/xml/0006-012.xml',
 'tlg0006.tlg013': 'public/xml/0006-013.xml',
 'tlg0006.tlg014': 'public/xml/0006-014.xml',
 'tlg0006.tlg015': 'public/xml/0006-015.xml',
 'tlg0006.tlg016': 'public/xml/0006-016.xml',
 'tlg0006.tlg017': 'public/xml/0006-017.xml',
 'tlg0006.tlg018': 'public/xml/0006-018.xml',
 'tlg0006.tlg019': 'public/xml/0006-019.xml',
 'tlg0006.tlg031': 'public/xml/0006-031.xml',
 'tlg0007.tlg001': 'public/xml/000

In [20]:
def lemmatized_sentences_by_workid(workid):
    if workid in workid_filename_dict.keys():
        fn = workid_filename_dict[workid]
        response = requests.get(base_url + fn)
        soup = BeautifulSoup(response.text, "html.parser")
        lemmata_sentences = []
        for sent in soup.find_all("sentence"):
            sentence_lemmata = []
            for w in sent.find_all("word"):
                try:
                    if w["postag"][0] in ["n", "v", "a"]:
                        sentence_lemmata.append(w["lemma"])
                except:
                    try:
                        sentence_lemmata.append(w["lemma"])
                    except:
                        pass

            lemmata_sentences.append(sentence_lemmata)

    else:
        lemmata_sentences = LAGT[LAGT["doc_id"]==workid]["lemmatized_sentences"].tolist()[0]
    return lemmata_sentences

In [21]:
# test with individual document
lem_sents = lemmatized_sentences_by_workid("tlg2042.tlg001")

In [22]:
%%time
new_lemmatized_sentences = LAGT["doc_id"].apply(lemmatized_sentences_by_workid)

CPU times: user 7min 18s, sys: 8.26 s, total: 7min 26s
Wall time: 14min 30s


In [23]:
LAGT["lemmatized_sentences"] = new_lemmatized_sentences

In [24]:
def in_glaux(workid):
    return workid in workid_filename_dict.keys()
LAGT["glaux?"] = LAGT["doc_id"].apply(in_glaux)

In [25]:
LAGT["glaux?"].sum()

912

In [26]:
LAGT[LAGT["glaux?"]].head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,glaux?
12,tlg0015,tlg0015.tlg001,tlg0015.tlg001.1st1K-grc1.xml,Herodian,Ab excessu divi Marci,\nΟἱ πλεῖστοι τῶν περὶ συγκομιδὴν ἱστορίας ἀσχ...,46751,1Kgr,"[[πολύς, συγκομιδή, ἱστορία, ἀσχολέω, ἔργον, γ...",True
87,tlg0057,tlg0057.tlg001,tlg0057.tlg001.1st1K-grc1.xml,Galen,Adhortatio ad artes addiscendas,Εἰ μὲν μηδόλως λόγου μέτεστι τοῖς ἀλόγοις \n ...,5038,1Kgr,"[[λόγος, μέτειμι, ἄλογος, ὀνομάζω, ζῷον, ἄδηλο...",True
88,tlg0057,tlg0057.tlg001,tlg0057.tlg001.1st1K-grc2.xml,Galen,Protrepticus,\nΕἰ μὲν μηδ’ ὅλως λόγου μέτεστι τοῖς ἀλόγοις\...,4975,1Kgr,"[[λόγος, μέτειμι, ἄλογος, ὀνομάζω, ζῷον, ἄδηλο...",True
89,tlg0057,tlg0057.tlg002,tlg0057.tlg002.1st1K-grc1.xml,Galen,De optima doctrina,\nΤὴν εἰς ἑκάτερα ἐπιχείρησιν ἀρίστην εἶναι\n\...,1828,1Kgr,"[[ἑκάτερος, ἐπιχείρησις, ἀγαθός, εἰμί, διδασκα...",True
90,tlg0057,tlg0057.tlg003,tlg0057.tlg003.1st1K-grc1.xml,Galen,Quod optimus medicus sit quoque philosophus,"Οἷόν τι πεπόνθασιν οἱ πολλοὶ τῶν ἀθλητῶν, ἐπιθ...",1382,1Kgr,"[[τις, πάσχω, πολύς, ἀθλητής, ἐπιθυμέω, ὀλυμπι...",True


In [27]:
def get_lemmata_n(lemmatized_sentences):
    try:
        return len([l for sent in lemmatized_sentences for l in sent])
    except:
        return None
LAGT["lemmata_n"] = LAGT["lemmatized_sentences"].apply(get_lemmata_n)

In [29]:
LAGT[LAGT["glaux?"]].head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,glaux?,lemmata_n
12,tlg0015,tlg0015.tlg001,tlg0015.tlg001.1st1K-grc1.xml,Herodian,Ab excessu divi Marci,\nΟἱ πλεῖστοι τῶν περὶ συγκομιδὴν ἱστορίας ἀσχ...,46751,1Kgr,"[[πολύς, συγκομιδή, ἱστορία, ἀσχολέω, ἔργον, γ...",True,25832.0
87,tlg0057,tlg0057.tlg001,tlg0057.tlg001.1st1K-grc1.xml,Galen,Adhortatio ad artes addiscendas,Εἰ μὲν μηδόλως λόγου μέτεστι τοῖς ἀλόγοις \n ...,5038,1Kgr,"[[λόγος, μέτειμι, ἄλογος, ὀνομάζω, ζῷον, ἄδηλο...",True,2619.0
88,tlg0057,tlg0057.tlg001,tlg0057.tlg001.1st1K-grc2.xml,Galen,Protrepticus,\nΕἰ μὲν μηδ’ ὅλως λόγου μέτεστι τοῖς ἀλόγοις\...,4975,1Kgr,"[[λόγος, μέτειμι, ἄλογος, ὀνομάζω, ζῷον, ἄδηλο...",True,2619.0
89,tlg0057,tlg0057.tlg002,tlg0057.tlg002.1st1K-grc1.xml,Galen,De optima doctrina,\nΤὴν εἰς ἑκάτερα ἐπιχείρησιν ἀρίστην εἶναι\n\...,1828,1Kgr,"[[ἑκάτερος, ἐπιχείρησις, ἀγαθός, εἰμί, διδασκα...",True,943.0
90,tlg0057,tlg0057.tlg003,tlg0057.tlg003.1st1K-grc1.xml,Galen,Quod optimus medicus sit quoque philosophus,"Οἷόν τι πεπόνθασιν οἱ πολλοὶ τῶν ἀθλητῶν, ἐπιθ...",1382,1Kgr,"[[τις, πάσχω, πολύς, ἀθλητής, ἐπιθυμέω, ὀλυμπι...",True,712.0


In [32]:
(LAGT[LAGT["glaux?"]]["lemmata_n"] / LAGT[LAGT["glaux?"]]["wordcount"]).max()

9.831325301204819

# Pedalion trees data

In [33]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("perseids-publications").get_repo("pedalion-trees" )
paths = []
contents = repo.get_contents("public/xml")
while contents:
    file_content = contents.pop(0)
    path_and_name = file_content.path
    if ".xml" in str(path_and_name):
        paths.append(path_and_name)

CPU times: user 17.3 ms, sys: 7.32 ms, total: 24.6 ms
Wall time: 1.18 s


In [34]:
paths[0]

'public/xml/achar.xml'

In [35]:
base_url = "https://raw.githubusercontent.com/perseids-publications/pedalion-trees/master/"

In [36]:
def lemmatized_sentences_by_path(path, base_url=base_url):
    response = requests.get(base_url + path)
    soup = BeautifulSoup(response.text, "html.parser")
    lemmata_sentences = []
    doc_ids = []
    for sent in soup.find_all("sentence"):
        doc_ids.append(sent['document_id'])
        sentence_lemmata = []
        for w in sent.find_all("word"):
            try:
                if w["postag"][0] in ["n", "v", "a"]:
                    sentence_lemmata.append(w["lemma"])
            except:
                try:
                    sentence_lemmata.append(w["lemma"])
                except:
                    pass
        lemmata_sentences.append(sentence_lemmata)
    return doc_ids, lemmata_sentences

In [37]:
docs_ids, docs_lemmatized_sentences = [], []
for path in paths:
    doc_ids, lemmatized_sentences = lemmatized_sentences_by_path(path, base_url)
    docs_ids.append(doc_ids)
    docs_lemmatized_sentences.append(lemmatized_sentences)

In [38]:
docs_lemmatized_sentences[10][:10]

[['ὀφείλω',
  'Ἀργώ',
  'διαπέταμαι',
  'σκάφος',
  'Κόλχος',
  'αἶα',
  'κυάνεος',
  'Συμπληγάς',
  'νάπη',
  'Πήλιον',
  'πίπτω',
  'τέμνω',
  'πεύκη',
  'ἐρετμόω',
  'χήρ',
  'ἀνήρ',
  'ἀριστεύς',
  'παγχρύσεος',
  'δέρος',
  'Πελίας',
  'μετέρχομαι'],
 ['δέσποινα',
  'ἐμός',
  'Μήδεια',
  'πύργος',
  'γῆ',
  'πλέω',
  'Ἰώλκιος',
  'ἔρως',
  'θυμός',
  'ἐκπλήσσω',
  'Ἰάσων'],
 ['κτείνω',
  'πείθω',
  'Πελιάς',
  'κόρη',
  'πατήρ',
  'κατοικέω',
  'γῆ',
  'Κορίνθιος',
  'φίλος',
  'ἀμπλακεῖν',
  'πάτρα'],
 ['ἔχω',
  'μεμπτός',
  'βίος',
  'ἀνήρ',
  'τέκνον',
  'ἁνδάνω',
  'φυγάς',
  'πολίτης',
  'ἀφικνέομαι',
  'χθών',
  'πᾶς',
  'συμφέρω',
  'Ἰάσων'],
 ['μέγας', 'γίγνομαι', 'σωτηρία', 'γυνή', 'ἀνήρ', 'διχοστατέω'],
 ['ἐχθρός', 'πᾶς', 'νοσέω', 'φίλος'],
 ['προδίδωμι',
  'τέκνον',
  'δεσπότις',
  'ἐμός',
  'γάμος',
  'Ἰάσων',
  'βασιλικός',
  'εὐνάζω',
  'γαμέω',
  'Κρέων',
  'παῖς',
  'αἰσυμνάω',
  'χθών'],
 ['Μήδεια',
  'δύστηνος',
  'ἀτιμάζω',
  'βοάω',
  'ὅρκος',
  'ἀνακαλέω',
  '

In [39]:
[set(ids) for ids in docs_ids]

[{'0019-001'},
 {'0058-001'},
 {'0096-00#', '0096-002'},
 {'1220-001'},
 {'0554-001'},
 {'0003-001',
  '0006-003',
  '0007-004',
  '0007-086',
  '0008-001',
  '0011-001',
  '0011-002',
  '0011-003',
  '0011-004',
  '0011-005',
  '0012-001',
  '0012-002',
  '0014-018',
  '0016-001',
  '0019-008',
  '0020-002',
  '0026-001',
  '0032-001',
  '0032-007',
  '0059-001',
  '0060-001',
  '0062-012',
  '0062-014',
  '0081-001',
  '0085-001',
  '0085-002',
  '0085-004',
  '0085-005',
  '0085-006',
  '0085-007',
  '0096-002',
  '0526-004',
  '0540-001',
  '0540-012',
  '0540-013',
  '0540-014',
  '0540-019',
  '0540-024',
  '0541-007',
  '0543-001',
  '1220-001',
  'Chilia',
  'Genesis',
  'Genesis_deel2',
  'NT',
  'PaeanBrev',
  'Pedalion',
  'Pedalion2',
  'Pedalion3',
  'Pedalion4',
  'Ps-Luc'},
 {'0041-001'},
 {'0059-003'},
 {'0557-001'},
 {'0537-012'},
 {'0006-003'},
 {'Chilia', 'Pedalion', 'Pedalion2', 'Pedalion3', 'Pedalion4'},
 {'0003-001',
  '0006-003',
  '0007-004',
  '0007-015',
  '00

In [ ]:
ids_sents_dict = {}
for ids, sents in zip(docs_ids, docs_lemmatized_sentences):
    if len(set(ids))==1:
        ids_sents_dict[ids[0]] = sents

In [ ]:
len(ids_sents_dict)

In [ ]:
ids_sents_dict

In [43]:
def add_pedalion_lemmata(doc_id, lemmatized_sentences):
    if lemmatized_sentences == None:
        doc_id = doc_id.replace("tlg", "").replace(".", "-")
        if doc_id in ids_sents_dict.keys():
            lemmatized_sentences = ids_sents_dict[doc_id]
    return lemmatized_sentences

In [44]:
lagt_lemmatized_sentences = LAGT.apply(lambda row: add_pedalion_lemmata(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [45]:
LAGT["lemmatized_sentences"] = lagt_lemmatized_sentences

In [46]:
LAGT["lemmatized_sentences"].notnull().sum()

919

In [49]:
LAGT["pedalion?"] = LAGT["lemmatized_sentences"].notnull() & ~LAGT["glaux?"]

In [56]:
LAGT[LAGT["pedalion?"]]

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,glaux?,lemmata_n,pedalion?
80,tlg0041,tlg0041.tlg001,tlg0041.tlg001.1st1K-grc1.xml,Chionis Epistulae,Epistulae,Ἀπεδωκέ μοι Λῦσις τὴν ἐπιστολὴν τρίτην ἡμέρα...,4772,1Kgr,"[[Χίων, Μᾶτρις, χαίρω], [ἀποδίδωμι, Λῦσις, ἐπι...",False,NaN,True
421,tlg0537,tlg0537.tlg012,tlg0537.tlg012.1st1K-grc1.xml,Epicurus,Epistula Tertia ad Menoeceum,"Μήτε νέος τις ὢν μελλέτω φιλοσοφεῖν, μήτε γέρω...",1332,1Kgr,"[[Ἐπίκουρος, Μενοικεύς, χαίρω, ], [νέος, εἰμί,...",False,NaN,True
430,tlg0544,tlg0544.tlg002,tlg0544.tlg002.1st1K-grc1.xml,Sextus Empiricus,Adversus mathematicos,Τὴν πρὸς τοὺς ἀπὸ τῶν μαθημάτων ἀντίρρησιν κο...,189347,1Kgr,"[[μάθημα, ἀντίρρησις, διατίθημι, δοκέω, Ἐπίκου...",False,NaN,True
604,tlg1220,tlg1220.tlg001,tlg1220.tlg001.1st1K-grc1.xml,Batrachomyomachia,Batrachomyomachia,Ἀρχόμενος πρώτης σελίδος χορὸν ἐξ Ἑλικῶνος\nἐλ...,5426,1Kgr,"[[ἄρχω, πρῶτος, σελίς, χορός, Ἑλικών, ἔρχομαι,...",False,NaN,True
1164,tlg0006,tlg0006.tlg003,tlg0006.tlg003.perseus-grc2.xml,Euripides,Μήδεια,Εἴθ’ ὤφελ’ Ἀργοῦς μὴ διαπτάσθαι σκάφος Κόλχων...,8026,perseus,"[[ὀφείλω, Ἀργώ, διαπέταμαι, σκάφος, Κόλχος, αἶ...",False,NaN,True
1938,tlg2003,tlg2003.tlg001,tlg2003.tlg001.perseus-grc1.xml,Julian the Emperor,Panegyric in Honor of the Emperor Constantinus,"\n*pa/lai me proqumou/menon, w)= me/giste basi...",11584,perseus,"[[προθυμέομαι, μέγας, βασιλεύς, σός, ἀρετή, πρ...",False,NaN,True
1958,tlg4029,tlg4029.tlg001,tlg4029.tlg001.perseus-grc1.xml,Procopius,de Bellis,Προκόπιος Καισαρεὺς τοὺς πολέμους ξυνέγραψεν ο...,224514,perseus,"[[Προκόπιος, Καισαρεύς, πόλεμος, συγγράφω, Ἰου...",False,NaN,True


# Add Gorman treebanks data

In [50]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("perseids-publications").get_repo("gorman-trees" )
paths = []
contents = repo.get_contents("public/xml")
while contents:
    file_content = contents.pop(0)
    path_and_name = file_content.path
    if ".xml" in str(path_and_name):
        paths.append(path_and_name)

CPU times: user 25.6 ms, sys: 6.51 ms, total: 32.1 ms
Wall time: 1.22 s


In [51]:
paths

['public/xml/aeschines-1-1-50-bu1.xml',
 'public/xml/aeschines-1-101-150-bu1.xml',
 'public/xml/aeschines-1-151-196-bu1.xml',
 'public/xml/aeschines-1-51-100-bu1.xml',
 'public/xml/antiphon-1-bu2.xml',
 'public/xml/antiphon-2-bu2.xml',
 'public/xml/antiphon-5-bu2.xml',
 'public/xml/antiphon-6-bu2.xml',
 'public/xml/appian-bc-1-0-1-4-bu1.xml',
 'public/xml/appian-bc-1-11-14-bu1.xml',
 'public/xml/appian-bc-1-5-7-bu1.xml',
 'public/xml/appian-bc-1-8-10-bu1.xml',
 'public/xml/aristotle-politics-book-1-bu1.xml',
 'public/xml/aristotle-politics-book-2-bu2.xml',
 'public/xml/athen12-1-9-2019.xml',
 'public/xml/athen12-10-19-2019.xml',
 'public/xml/athen12-20-29-2019.xml',
 'public/xml/athen12-20-29-jan-15.xml',
 'public/xml/athen12-30-39-2019.xml',
 'public/xml/athen12-30-39-jan-15.xml',
 'public/xml/athen12-40-49-2019.xml',
 'public/xml/athen12-40-49-jan-15.xml',
 'public/xml/athen12-50-59-2019.xml',
 'public/xml/athen12-50-59-jan-15.xml',
 'public/xml/athen12-60-69-2019.xml',
 'public/xml/

In [52]:
base_url = "https://raw.githubusercontent.com/perseids-publications/gorman-trees/master/"

def lemmatized_sentences_by_path(path, base_url=base_url):
    response = requests.get(base_url + path)
    soup = BeautifulSoup(response.text, "html.parser")
    lemmata_sentences = []
    doc_paths = []
    doc_subdocs = []
    doc_ids = []
    for sent in soup.find_all("sentence"):
        doc_paths.append(path)
        doc_ids.append(sent['document_id'])
        doc_subdocs.append(sent['id'])
        sentence_lemmata = []
        for w in sent.find_all("word"):
            try:
                if w["postag"][0] in ["n", "v", "a"]:
                    sentence_lemmata.append(w["lemma"])
            except:
                try:
                    sentence_lemmata.append(w["lemma"])
                except:
                    pass
        lemmata_sentences.append(sentence_lemmata)
    return (doc_paths, doc_ids, doc_subdocs, lemmata_sentences)

In [53]:
docs_paths, docs_ids, docs_subdocs, docs_lemmatized_sentences = [], [], [], []

for path in paths:
    doc_paths, doc_ids, doc_subdocs, lemmatized_sentences = lemmatized_sentences_by_path(path, base_url)
    docs_paths.extend(doc_paths)
    docs_ids.extend(doc_ids)
    docs_subdocs.extend(doc_subdocs)
    docs_lemmatized_sentences.extend(lemmatized_sentences)

In [230]:
gorman_df = pd.DataFrame([docs_paths, docs_ids, subdocs, docs_lemmatized_sentences]).T
gorman_df.head(5)

,0,1,2,3
0,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,1,"[πολίτης, ἀνήρ, Ἀθηναῖος, γραφής, γράφω, εὔθυν..."
1,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,2,"[οἶδα, ἔνοχος, εἰμί, ὀλίγος, πρότερος, ἀκούω, ..."
2,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,2,"[ἔοικα, ἀνήρ, Ἀθηναῖος, ἔθω, λόγος, λέγω3, δημ..."
3,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,2,"[ἴδιος, ἔχθρα, πολύς, κοινός, ἐπανορθόω]"
4,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,3,"[ὅλος, ἀγών, φαίνω, πόλις, αἴτιος, εἰμί, Τίμαρ..."


In [231]:
gorman_df.columns = ["path_raw", "doc_id_raw", "subdoc", "sentence"]

In [232]:
gorman_df["path"] = gorman_df["path_raw"].apply(lambda x: x.rpartition("/")[2])

In [233]:
gorman_df["doc_id_raw"].unique().tolist()

['urn:cts:greekLit:tlg0026.tlg001.perseus-grc1',
 'http://perseids.org/annotsrc/urn:cts:greekLit:tlg0028.tlg001.perseus-grc1',
 'urn:cts:greekLit:tlg0028.tlg002.perseus-grc1',
 'urn:cts:greekLit:tlg0028.tlg005.perseus-grc1',
 'urn:cts:greekLit:tlg0028.tlg006.perseus-grc1',
 'urn:cts:greekLit:tlg0551.tlg017.perseus-grc2',
 'urn:cts:greekLit:tlg0086.tlg035.perseus-grc1:1',
 'urn:cts:greekLit:tlg0086.tlg035.perseus-grc1:2',
 'urn:cts:greekLit:tlg0008.tlg001.perseus-grc1',
 'Perseus:text:2008.01.0405',
 'urn:cts:greekLit:tlg0014.tlg059.perseus-grc2',
 '0014-059',
 '0014-001',
 'urn:cts:greekLit:tlg0014.tlg018.perseus-grc1',
 'urn:cts:greekLit:tlg0014.tlg004.perseus-grc1',
 '0014-046',
 '0014-047',
 '0014-049',
 '0014-050',
 '0014-052',
 '0014-053',
 'http://perseids.org/annotsrc/urn:cts:greekLit:tlg0060.tlg001.perseus-grc3',
 'urn:cts:greekLit:tlg0060.tlg001.perseus-grc3',
 'urn:cts:greekLit:tlg0081.tlg001.perseus-grc1',
 'urn:cts:greekLit:tlg0016.tlg001.perseus-grc1',
 'urn:cts:greekLit:t

In [234]:
def find_match(x):
    doc_id = None
    match = re.search("tlg\d{4}\.tlg\d{3}", x)
    if match:
        doc_id = match.group()
    else:
        match = re.search("(\d{4})\-(\d{3})", x)
        if match:
            doc_id = "tlg" + match.groups()[0] + ".tlg" + match.groups()[1]
    return doc_id

gorman_df["doc_id"] = gorman_df["doc_id_raw"].apply(find_match)

In [235]:
gorman_df.sort_values("path")["path"].unique()

array(['aeschines-1-1-50-bu1.xml', 'aeschines-1-101-150-bu1.xml',
       'aeschines-1-151-196-bu1.xml', 'aeschines-1-51-100-bu1.xml',
       'antiphon-1-bu2.xml', 'antiphon-2-bu2.xml', 'antiphon-5-bu2.xml',
       'antiphon-6-bu2.xml', 'appian-bc-1-0-1-4-bu1.xml',
       'appian-bc-1-11-14-bu1.xml', 'appian-bc-1-5-7-bu1.xml',
       'appian-bc-1-8-10-bu1.xml', 'aristotle-politics-book-1-bu1.xml',
       'aristotle-politics-book-2-bu2.xml', 'athen12-1-9-2019.xml',
       'athen12-10-19-2019.xml', 'athen12-20-29-2019.xml',
       'athen12-20-29-jan-15.xml', 'athen12-30-39-2019.xml',
       'athen12-30-39-jan-15.xml', 'athen12-40-49-2019.xml',
       'athen12-40-49-jan-15.xml', 'athen12-50-59-2019.xml',
       'athen12-50-59-jan-15.xml', 'athen12-60-69-2019.xml',
       'athen12-60-69-jan-15.xml', 'athen12-70-81-2019.xml',
       'athen12-70-81-jan-15.xml', 'athen13-1-9-2019.xml',
       'athen13-1-9-jan-15.xml', 'athen13-10-19-2019.xml',
       'athen13-10-19-jan-15.xml', 'athen13-20-29-

In [236]:
docs_paths_dict = {}
for doc_id in gorman_df["doc_id"].unique():
    docs_paths_dict[doc_id] = gorman_df[gorman_df["doc_id"]==doc_id]["path"].unique().tolist()

In [237]:
docs_paths_dict

{'tlg0026.tlg001': ['aeschines-1-1-50-bu1.xml',
  'aeschines-1-101-150-bu1.xml',
  'aeschines-1-151-196-bu1.xml',
  'aeschines-1-51-100-bu1.xml'],
 'tlg0028.tlg001': ['antiphon-1-bu2.xml'],
 'tlg0028.tlg002': ['antiphon-2-bu2.xml'],
 'tlg0028.tlg005': ['antiphon-5-bu2.xml'],
 'tlg0028.tlg006': ['antiphon-6-bu2.xml'],
 'tlg0551.tlg017': ['appian-bc-1-0-1-4-bu1.xml',
  'appian-bc-1-11-14-bu1.xml',
  'appian-bc-1-5-7-bu1.xml',
  'appian-bc-1-8-10-bu1.xml'],
 'tlg0086.tlg035': ['aristotle-politics-book-1-bu1.xml',
  'aristotle-politics-book-2-bu2.xml'],
 'tlg0008.tlg001': ['athen12-1-9-2019.xml',
  'athen12-10-19-2019.xml',
  'athen12-20-29-2019.xml',
  'athen12-20-29-jan-15.xml',
  'athen12-30-39-2019.xml',
  'athen12-30-39-jan-15.xml',
  'athen12-40-49-2019.xml',
  'athen12-40-49-jan-15.xml',
  'athen12-50-59-2019.xml',
  'athen12-50-59-jan-15.xml',
  'athen12-60-69-2019.xml',
  'athen12-60-69-jan-15.xml',
  'athen12-70-81-2019.xml',
  'athen12-70-81-jan-15.xml',
  'athen13-1-9-2019.xml'

In [240]:
for_manual_sorting = ""
for key, val in docs_paths_dict.items():
    
    if len(val)>1:
        for_manual_sorting += "\ndocs_paths_dict[\"" + key + "\"] = " + str(sorted(val))

In [241]:
print(for_manual_sorting)


docs_paths_dict["tlg0026.tlg001"] = ['aeschines-1-1-50-bu1.xml', 'aeschines-1-101-150-bu1.xml', 'aeschines-1-151-196-bu1.xml', 'aeschines-1-51-100-bu1.xml']
docs_paths_dict["tlg0551.tlg017"] = ['appian-bc-1-0-1-4-bu1.xml', 'appian-bc-1-11-14-bu1.xml', 'appian-bc-1-5-7-bu1.xml', 'appian-bc-1-8-10-bu1.xml']
docs_paths_dict["tlg0086.tlg035"] = ['aristotle-politics-book-1-bu1.xml', 'aristotle-politics-book-2-bu2.xml']
docs_paths_dict["tlg0008.tlg001"] = ['athen12-1-9-2019.xml', 'athen12-10-19-2019.xml', 'athen12-20-29-2019.xml', 'athen12-20-29-jan-15.xml', 'athen12-30-39-2019.xml', 'athen12-30-39-jan-15.xml', 'athen12-40-49-2019.xml', 'athen12-40-49-jan-15.xml', 'athen12-50-59-2019.xml', 'athen12-50-59-jan-15.xml', 'athen12-60-69-2019.xml', 'athen12-60-69-jan-15.xml', 'athen12-70-81-2019.xml', 'athen12-70-81-jan-15.xml', 'athen13-1-9-2019.xml', 'athen13-1-9-jan-15.xml', 'athen13-10-19-2019.xml', 'athen13-10-19-jan-15.xml', 'athen13-20-29-2019.xml', 'athen13-20-29-jan-15.xml', 'athen13-30-

In [243]:
docs_paths_dict["tlg0026.tlg001"] = ['aeschines-1-1-50-bu1.xml', 'aeschines-1-51-100-bu1.xml', 'aeschines-1-101-150-bu1.xml', 'aeschines-1-151-196-bu1.xml']
docs_paths_dict["tlg0551.tlg017"] = ['appian-bc-1-0-1-4-bu1.xml', 'appian-bc-1-5-7-bu1.xml', 'appian-bc-1-8-10-bu1.xml', 'appian-bc-1-11-14-bu1.xml']
docs_paths_dict["tlg0086.tlg035"] = ['aristotle-politics-book-1-bu1.xml', 'aristotle-politics-book-2-bu2.xml']
docs_paths_dict["tlg0008.tlg001"] = ['athen12-1-9-2019.xml', 'athen12-10-19-2019.xml', 'athen12-20-29-2019.xml', 'athen12-30-39-2019.xml', 'athen12-40-49-2019.xml', 'athen12-50-59-2019.xml', 'athen12-60-69-2019.xml', 'athen12-70-81-2019.xml', 'athen13-1-9-2019.xml', 'athen13-10-19-2019.xml', 'athen13-20-29-2019.xml', 'athen13-30-39-2019.xml',  'athen13-40-49-2019.xml', 'athen13-50-59-2019.xml', 'athen13-60-69-2019.xml', 'athen13-70-79-2019.xml',  'athen13-80-89-2019.xml', 'athen13-90-95-2019.xml']
docs_paths_dict["tlg0014.tlg018"] = ['demosthenes-18-1-50-bu2.xml', 'demosthenes-18-51-100-bu1.xml', 'demosthenes-18-101-150-bu2.xml', 'demosthenes-18-151-200-bu2.xml', 'demosthenes-18-201-275-bu1.xml', 'demosthenes-18-276-324-bu1.xml']
docs_paths_dict["tlg0060.tlg001"] = ['diodsic-11-1-20-bu4.xml', 'diodsic11-21-40-bu2.xml', 'diodsic11-41-60-bu1.xml', 'diodsic11-61-80-bu1.xml', 'diodsic-11-81-92-bu1.xml']
docs_paths_dict["tlg0081.tlg001"] = ['dion-hal-1-1-15-bu2.xml', 'dion-hal-1-16-30-bu1.xml', 'dion-hal-1-31-45-bu1.xml', 'dion-hal-1-46-60-bu1.xml', 'dion-hal-1-61-75-bu1.xml', 'dion-hal-1-76-90-bu1.xml']
docs_paths_dict["tlg0016.tlg001"] = ['hdt-1-1-19-bu3-2019.xml', 'hdt-1-20-39-bu2-2019.xml', 'hdt-1-40-59-bu2-2019.xml', 'hdt-1-60-79-bu2-2019.xml', 'hdt-1-80-99-bu5-2019.xml', 'hdt-1-100-119-bu3-2019.xml', 'hdt-1-120-149-bu2-2019.xml', 'hdt-1-150-169-bu3-2019.xml', 'hdt-1-170-189-bu2-2019.xml', 'hdt-1-190-216-bu2-2019.xml']
docs_paths_dict["tlg0526.tlg004"] = ['josephus-bj-1-1-2-bu1.xml',  'josephus-bj-1-3-5-bu2.xml', 'josephus-bj-1-6-10-bu1.xml', 'josephus-bj-1-11-15-bu1.xml', 'josephus-bj-1-16-20-bu1.xml', 'josephus-bj-1-21-25-bu1.xml']
docs_paths_dict["tlg0007.tlg015"] = ['plut-alcib-1-17-bu1.xml', 'plut-alcib-18-39-bu1.xml']
docs_paths_dict["tlg0007.tlg004"] = ['plutarch-lycurgus-1-15-bu4.xml', 'plutarch-lycurgus-16-31-bu2.xml']
#docs_paths_dict["tlg0543.tlg001"] = ['polybius1-1-9-2017.xml', 'polybius1-10-19-2017.xml', 'polybius1-20-29-2017.xml', 'polybius1-30-39-2017.xml', 'polybius1-40-49-2017.xml', 'polybius1-50-59-2017.xml', 'polybius1-60-69-2017.xml', 'polybius1-70-79-2017.xml', 'polybius1-80-88-2017.xml', 'polybius-2-1-10-bu1.xml', 'polybius-2-11-20-bu1.xml', 'polybius-2-21-30-bu2.xml', 'polybius-2-31-40-bu2.xml', 'polybius-2-41-50-bu1.xml', 'polybius-2-51-60-bu1.xml', 'polybius-2-61-71-bu2.xml', 'polybius-6-2-15-bu1.xml', 'polybius-6-16-30-bu1.xml', 'polybius-6-31-45-bu1.xml', 'polybius-6-46-58-bu1.xml', 'polybius-9-1-20-bu1.xml', 'polybius-9-21-33-bu1.xml', 'polybius-9-34-45-bu1.xml','polybius-10-1-10-bu1.xml', 'polybius-10-11-20-bu1.xml', 'polybius-10-21-35-bu2.xml', 'polybius-10-36-49-bu1.xml' 'polybius-21-1-10-bu1.xml', 'polybius-21-11-20-bu1.xml', 'polybius-21-21-30-bu1.xml', 'polybius-21-31-47-bu1.xml']
#docs_paths_dict["tlg0003.tlg001"] = ['thuc-1-1-20-bu5.xml', 'thuc-1-21-40-bu4.xml', 'thuc-1-41-60-bu3.xml', 'thuc-1-61-80-bu3.xml', 'thuc-1-81-100-bu2.xml','thuc-1-101-120-bu2.xml', 'thuc-1-121-146-bu3.xml' 'thuc-3-1-20-bu1.xml', 'thuc-3-21-40-bu1.xml']
docs_paths_dict["tlg0032.tlg007"] = ['xen-cyr-1-1-2-bu1.xml', 'xen-cyr-1-3-4-bu1.xml', 'xen-cyr-1-5-bu1.xml', 'xen-cyr-1-6-bu1.xml', 'xen-cyr-7-1-3-tree.xml', 'xen-cyr-7-4-5-tree.xml', 'xen-cyr-8-1-8-4-bu1.xml', 'xen-cyr-8-5-7-bu1.xml', 'xen-cyr-8-8-bu1.xml']
docs_paths_dict["tlg0032.tlg001"] = ['xen-hell-1-1-4-bu2.xml', 'xen-hell-1-5-7-bu1.xml', 'xen-hell-2-bu1.xml', 'xen-hell-3-bu1.xml']

In [263]:
LAGT[LAGT["doc_id"].isin(docs_paths_dict.keys())].drop(["string"]).head(20)

In [249]:
def get_gorman_sentences(doc_id, lemmatized_sentences):
    if lemmatized_sentences == None:
        if doc_id in docs_paths_dict.keys():
            key = doc_id
            lemmatized_sentences = []
            subset = gorman_df[gorman_df["doc_id"]==key]
            for path in docs_paths_dict[key]:
                lemmatized_sentences.extend(subset[subset["path"]==path].sort_values("subdoc")["sentence"].tolist())
    return lemmatized_sentences

In [254]:
lagt_lemmatized_sentences = LAGT.apply(lambda row: get_gorman_sentences(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [257]:
LAGT["lemmatized_sentences"] = lagt_lemmatized_sentences
LAGT["lemmatized_sentences"].notnull().sum()

923

In [55]:
LAGT["gorman?"] = LAGT["lemmatized_sentences"].notnull() & ~LAGT["glaux?"] & ~LAGT["pedalion?"]

KeyError: 'GLAUx?'

In [54]:
LAGT[LAGT["gorman?"]]

KeyError: 'gorman?'

### AGDT

In [ ]:
###
https://github.com/PerseusDL/treebank_data/zipball/master